In [1]:
# vị trí chưa file data - sau tên file phải có đuôi XLSX
Link_Data = r'G:\Other computers\My Laptop\Desktop\KOMPA\data\NOVALAND\october\data.xlsx'
# vị trí chứa file key- sau tên file phải có đuôi XLSX
Link_key = r'G:\Other computers\My Laptop\Desktop\KOMPA\data\VINGROUP\key_novagroup.xlsx'
### nếu data trên 50k buzz thì hãy chọn link2 là link 1 file khác
Link2 = r'G:\Other computers\My Laptop\Desktop\KOMPA\data\NOVALAND\october\kq.xlsx'


In [2]:
### ko sửa đổi bất cứ thông tin gì ở dưới
import pandas as pd
from openpyxl.workbook import Workbook
wb = Workbook()
#wb.create_sheet
#wb.save(filename = 'sample_book.xlsx')
import openpyxl
import numpy as np

def text_modify(text_input):
    text = str(text_input).lower()
    return text

def isspam (text):
    result = False
    if text == "no data":
        result = True
    else:
        for i in range(0,len(text)):
            if text[i].isalnum() :
                result = False
                break
            else:
                if i == (len(text)-1):
                    result = True
    return result


data = pd.read_excel(Link_Data,sheet_name="Data")
Title_key = pd.read_excel(Link_key,sheet_name="Title")
Content_key = pd.read_excel(Link_key,sheet_name="Content")
SiteID_key = pd.read_excel(Link_key,sheet_name="SiteID")
Parent_ID_key = pd.read_excel(Link_key,sheet_name="ParentID")
Fanpage_key = pd.read_excel(Link_key,sheet_name="Fanpage")
# lấy data
SiteID_key = list(SiteID_key["SiteID"])
Parent_ID_key = list(Parent_ID_key["ParentID"])

Title_dict= {}
Content_dict = {}
list_channel= ["Facebook","News","Forum","Youtube","E-commerce","Social"]
for i in list_channel:
    Title_dict[i] = []
    Content_dict[i] = []
# dict key của title
for idx in Title_key.index:
    Channel = Title_key.loc[idx,"Channel"]
    Title_dict[Channel] += [str(Title_key.loc[idx,"Key_word"])]
# dict key của content    
for idx in Content_key.index:
    Channel = Content_key.loc[idx,"Channel"]
    Content_dict[Channel] += [str(Content_key.loc[idx,"Key_word"])]
    
# research spam youtube
data["Content"] = data["Content"].fillna("no data")
data["modi_content"] = data["Content"].apply(text_modify)
data["modi_title"] = data["Title"].apply(text_modify)
data["check"] = "None_check"
data["label_suggest"] = "None_mark"

# fanpage
Fanpage_key = list(Fanpage_key["ID"])
for idx in data.index:
    for key in Fanpage_key:
        if str(key) == data.loc[idx,"SiteId"]:
            data.loc[idx,"Channel"] = "Fanpage"
            data.loc[idx,"check"] = "checked"
            data.loc[idx,"label_suggest"] = "fanpage"
            


# xóa rác ký tự
for idx in data.index: 
    if data.loc[idx,"check"] == "None_check":
        content = data.loc[idx,"modi_content"]
        title = data.loc[idx,"modi_title"]
        channel = data.loc[idx,"Channel"]
        if isspam(content) == True:
            data.loc[idx,"label_suggest"] = "spam_ký tự"
            data.loc[idx,"check"] = "checked"


# xử lí siteid => parent id 
for idx in data.index: 
    if data.loc[idx,"check"] == "None_check":
        SiteID = data.loc[idx,"SiteId"]
        for key in SiteID_key:
            if str(SiteID) == str(key):
                data.loc[idx,"label_suggest"] = "spam_Siteid"
                data.loc[idx,"check"] = "checked"
                break
    if data.loc[idx,"check"] == "None_check":
        ParentID = data.loc[idx,"ParentId"]
        for key in Parent_ID_key:
            if str(ParentID) == str(key):
                data.loc[idx,"label_suggest"] = "spam_Siteid"
                data.loc[idx,"check"] = "checked"
                break
    


# xử lí title và content
for idx in data.index: 
    if data.loc[idx,"check"] == "None_check":
        title = data.loc[idx,"modi_title"]
        Channel = data.loc[idx,"Channel"]
        if Title_dict[Channel] == []:
            continue
        else:
            for key in Title_dict[Channel]:
                if str(key).lower() in title:
                    data.loc[idx,"label_suggest"] = "spam_Title"
                    data.loc[idx,"check"] = "checked"
                    break
    if data.loc[idx,"check"] == "None_check":
        content = data.loc[idx,"modi_content"]
        Channel = data.loc[idx,"Channel"]
        if Content_dict[Channel] == []:
            continue
        else:
            for key in Content_key:
                if str(key).lower() in content:
                    data.loc[idx,"label_suggest"] = "spam_Content"
                    data.loc[idx,"check"] = "checked"
                    break



# đánh dấu các label còn lại
for idx in data.index: 
    if data.loc[idx,"check"] == "None_check":
        data.loc[idx,"label_suggest"] = "other"
        data.loc[idx,"check"] = 'checked' 

'# xử lí siteid => parent id \nfor idx in data.index: \n    if data.loc[idx,"check"] == "None_check":\n        SiteID = data.loc[idx,"SiteId"]\n        for key in SiteID_key:\n            if str(SiteID) == str(key):\n                data.loc[idx,"label_suggest"] = "spam_Siteid"\n                data.loc[idx,"check"] = "checked"\n                break\n    if data.loc[idx,"check"] == "None_check":\n        ParentID = data.loc[idx,"ParentId"]\n        for key in Parent_ID_key:\n            if str(ParentID) == str(key):\n                data.loc[idx,"label_suggest"] = "spam_Siteid"\n                data.loc[idx,"check"] = "checked"\n                break\n    \n\n\n# xử lí title và content\nfor idx in data.index: \n    if data.loc[idx,"check"] == "None_check":\n        title = data.loc[idx,"modi_title"]\n        Channel = data.loc[idx,"Channel"]\n        if Title_dict[Channel] == []:\n            continue\n        else:\n            for key in Title_dict[Channel]:\n                if str(

In [3]:
data = data[['Id', 'TopicId',"label_suggest", 'Topic', 'Title', 'Content', 'Description',
       'UrlComment', 'PublishedDate', 'Sentiment', 'SiteName', 'SiteId',
       'Channel', 'Author', 'AuthorId', 'UrlTopic', 'ParentId', 'Labels1',
       'Labels2', 'Labels3', 'Labels4', 'Type', 'Level', 'Tags', 'Labels']]
data = data.set_index("Id")

In [4]:
writer = pd.ExcelWriter(Link2,engine="openpyxl",mode='a',if_sheet_exists="replace")
# write dataframe to excel sheet named 'youtube'
data.to_excel(writer, "Data_modified")
# save the excel file
writer.save()


print("Đã xử lý dữ liệu xong")

Đã xử lý dữ liệu xong
